In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import torch
from nn import MotifCaller, NaiveCaller
from training_data import data_preproc, load_training_data
from utils import get_savepaths
import numpy as np
from sklearn.preprocessing import normalize
from greedy_decoder import GreedyCTCDecoder
from Levenshtein import ratio
from utils import load_model, get_metrics_for_evaluation, sort_transcript
from sklearn.model_selection import train_test_split
from beam_search_decoder import beam_search_ctc
import torch.nn as nn
from tqdm.notebook import tqdm
import numpy as np
import matplotlib.pyplot as plt

In [31]:
n_classes = 19
model_path = r"C:\Users\Parv\Doc\HelixWorks\Basecalling\code\motifcaller\models\empirical\filtered_model.pth"
model_path = r"C:\Users\Parv\Doc\HelixWorks\Basecalling\code\motifcaller\models\empirical\filtered_6_motifs_30_epochs.pth"
labels_int = np.arange(n_classes).tolist()
labels = [f"{i}" for i in labels_int] # Tokens to be fed into greedy decoder
greedy_decoder = GreedyCTCDecoder(labels = labels)
ctc = nn.CTCLoss(blank=0, reduction='mean', zero_infinity=True)

In [32]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.set_default_device(device)
model = load_model(model_path=model_path, device=device, n_classes=n_classes)

dataset_path = r"C:\Users\Parv\Doc\HelixWorks\Basecalling\code\motifcaller\data\empirical\pickled_datasets\28_2_25.pkl"
test_dataset_path = r"C:\Users\Parv\Doc\HelixWorks\Basecalling\code\datasets\empirical\full_empirical_test_dataset_v5_payload_seq.pkl"
#dataset = pd.read_pickle(dataset_path)
#dataset = pd.read_pickle(test_dataset_path)

#X, y, payloads = load_training_data(
#        test_dataset_path, column_x='squiggle', column_y='motif_seq', payload=True, sampling_rate=0.2)

In [33]:
X, y, payloads = load_training_data(
        test_dataset_path, column_x='squiggle', column_y='Spacer_Sequence', payload=True, sampling_rate=0.2)

Index(['ONT_Barcode', 'HW_Address', 'Payload', 'Library_Motifs', 'read_id',
       'squiggle', 'Motifs', 'Spacer_Sequence', 'Payload_Sequence'],
      dtype='object')


In [34]:
X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42)

In [35]:
X_train, X_test, payloads_train, payloads_test = train_test_split(
        X, payloads, test_size=0.2, random_state=42)

In [36]:
def evaluate_prediction(prediction, original):

    found = 0
    err = 0
    for i, j in zip(prediction, original):
        for k in range(len(i)):
            if i[k] in j:
                found += 1
            else:
                err += 1

    return found, err

In [41]:

counter = 0
sum_diff = 0

for x, y, payload in tqdm(zip(X_test, y_test, payloads_test), total=len(X_test)):
    input_sequence = torch.tensor(
        x, dtype=torch.float32)
    input_sequence = input_sequence.view(
        1, 1, len(x)).to(device)
    model_output = model(input_sequence)
    model_output = model_output.permute(1, 0, 2)
    
    label_lengths = torch.tensor([len(y)])
    target_sequence = torch.tensor(y).to(device)

    
    n_timesteps = model_output.shape[0]
    input_lengths = torch.tensor([n_timesteps])
    
    model_output_flattened = model_output.view(
        model_output.shape[0] * model_output.shape[1], n_classes)

    loss = ctc(
        model_output, target_sequence, input_lengths, label_lengths)
    #print(loss.item())
    
    greedy_transcript = " ".join(greedy_decoder(model_output))
    beam_transcript = beam_search_ctc(
        model_output_flattened.detach().cpu(), beam_width=10)
    actual_transcript = " ".join([str(i) for i in y])

    decoded_prediction = sort_transcript(beam_transcript)
    search_prediction = sort_transcript(actual_transcript)
    original = sort_transcript(" ".join([str(i) for i in payload]))
    print(decoded_prediction)
    print(search_prediction)
    print(original)
    
    found_motifs_caller = evaluate_prediction(decoded_prediction, original)
    found_motifs_search = evaluate_prediction(search_prediction, original)
    print(found_motifs_caller)
    print(found_motifs_search)
    sum_diff += found_motifs_caller[0] - found_motifs_search[0]
    
    #greedy_ratio = ratio(greedy_transcript, actual_transcript)
    #beam_ratio = ratio(beam_transcript, actual_transcript)
    #sum_diff += beam_ratio - greedy_ratio
    #print()
    counter+=1
    print(sum_diff)

    if counter == 10:
        print(sum_diff)
        break

  0%|          | 0/2762 [00:00<?, ?it/s]

[[1], [1], [], [3], [1], [], [], [1]]
[[], [], [3, 6], [3], [6], [], [], []]
[[1, 5, 6, 8], [1, 2, 3, 7], [3, 4, 5, 6], [1, 3, 6, 8], [1, 3, 6, 7], [2, 3, 7, 8], [2, 3, 7, 8], [2, 6, 7, 8]]
(4, 1)
(4, 0)
0
[[1], [1], [], [1], [1], [], [], [1]]
[[], [4], [], [], [], [], [], []]
[[2, 3, 4, 5], [1, 2, 3, 4], [2, 4, 6, 7], [1, 3, 4, 5], [3, 5, 7, 8], [2, 3, 5, 6], [2, 3, 4, 7], [1, 3, 4, 8]]
(3, 2)
(1, 0)
2
[[1], [1], [], [3], [1], [], [], [1]]
[[], [], [5], [], [], [], [], []]
[[2, 4, 6, 7], [1, 2, 4, 7], [1, 2, 3, 5], [2, 3, 4, 5], [1, 3, 4, 5], [1, 3, 5, 7], [1, 2, 3, 6], [2, 4, 5, 8]]
(3, 2)
(1, 0)
4
[[1], [1], [], [3], [1], [], [], [1]]
[[], [], [5], [], [], [1], [], []]
[[2, 3, 6, 8], [3, 4, 5, 6], [2, 5, 6, 7], [1, 3, 5, 6], [1, 2, 4, 7], [1, 4, 5, 8], [3, 4, 7, 8], [1, 4, 7, 8]]
(3, 2)
(2, 0)
5
[[1], [1], [], [1], [1], [], [], [1]]
[[], [], [], [3], [], [], [6], []]
[[2, 6, 7, 8], [3, 5, 6, 8], [1, 3, 5, 8], [1, 3, 4, 7], [1, 6, 7, 8], [1, 3, 7, 8], [2, 3, 4, 6], [2, 4, 7, 8]]
(2, 

In [26]:
1 * 100 /32

3.125

In [38]:
sum_diff

71